In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import os
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil,os,os.path
from os import listdir
from os.path import isfile, join
from PIL import Image, ImageEnhance
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import os, subprocess, zipfile
from tqdm.autonotebook import tqdm

kings_to_country = {'Alexander III': 'Russia',
 'Anna Ioannovna': 'Russia',
 'Augustus III the Sas': 'Poland',
 'Aurelian': 'Roman Empire',
 'Catherine II': 'Russia',
 'Constantine the Great': 'Roman Empire',
 'Elizabeth': 'Russia',
 'Ferdinand II': 'Austria',
 'Ferdinand III': 'Austria',
 'Franz Joseph I': 'Austria',
 'Friedrich Wilhelm I': 'Germany',
 'Friedrich Wilhelm II': 'Germany',
 'Friedrich Wilhelm III': 'Germany',
 'Hadrian': 'Roman Empire',
 'John II Casimir': 'Poland',
 'John III Sobieski': 'Poland',
 'Karl VI': 'Austria',
 'Leopold I': 'Austria',
 'Leopold V': 'Austria',
 'Ludwig I': 'Germany',
 'Ludwig II': 'Germany',
 'Maximilian III Jose': 'Germany',
 'Neron': 'Roman Empire',
 'Nicholas II': 'Russia',
 'Peter I': 'Russia',
 'Philip I': 'Roman Empire',
 'Sigismund I Old': 'Poland',
 'Sigismund III': 'Poland',
 'Stephen Báthory': 'Poland',
 'Trajan': 'Roman Empire'}
kings_to_country_index ={0: 4, 1: 4, 2: 2, 3: 3, 4: 4, 5: 3, 6: 4, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 3, 14: 2, 15: 2, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1, 21: 1, 22: 3, 23: 4, 24: 4, 25: 3, 26: 2, 27: 2, 28: 2, 29: 3}

In [ ]:
!pip install tqdm lap
!pip install efficientnet_pytorch

In [ ]:
# Connet to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
SEED = 1000

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
def load_dataset_helper(download_dir, working_dir, datafile):
  if not os.path.exists(os.path.join(working_dir,datafile)):
        base_dir = '/content/work/'

        if os.path.isfile(os.path.join(base_dir,datafile)) == False: 
          !rsync -ah --progress '$download_dir'$datafile '$base_dir'
        else:
          print('already exists!')

        try:
          os.mkdir(working_dir)
        except OSError:
          print ("Creation of the directory %s failed" % working_dir)
        else:
          print ("Successfully created the directory %s " % working_dir)

        #with zipfile.ZipFile(os.path.join(base_dir,datafile)) as zf:
        #  for member in tqdm(zf.infolist(), desc='Extracting '):
        #      try:
        #        zf.extract(member, working_dir)
        #      except zipfile.error as e:
        #          pass
        path_to_zip = os.path.join(download_dir,datafile)
        print(path_to_zip)
        !unzip $path_to_zip -d $working_dir

        print("Directory " , working_dir ,  " Created ")
  else:    
        print("Directory " , working_dir ,  " already exists")


def load_dataset(first_datafile = 'coins_kings'):
    first_datafile += ".zip"
    #YOU NEED TO CHANGE THIS DOWNLOAD_DIR PATH FOR YOUR VALID PATH!
    download_dir = '/content/gdrive/My\ Drive/magisterka/'
    working_dir = '/content/work'
    #working_dir_task_2_3 = '/content/work/task_2_3'

    load_dataset_helper(download_dir, working_dir, first_datafile )


In [ ]:
!rm -R /content/work
file_type = 'coins_kings'
for i in range(5):
  file = 'folds_data_packages/' + file_type +"_fold"+str(i+1)
  load_dataset(file)


In [ ]:
class OneCoinSideDataset(torchvision.datasets.ImageFolder):
  def __init__(self, root, kings, transform):
    super(OneCoinSideDataset, self).__init__(root, transform)
    self.kings = kings

  def __getitem__(self, index):
    # override ImageFolder's method
    """
    Args:
      index (int): Index
    Returns:
      tuple: (sample, resnet, target) where target is class_index of the target class.
    """
    path, target = self.samples[index]
    sample = self.loader(path)
    if self.kings:
      sample = sample.crop((0, 0, sample.width/2, sample.height))
    else:
      sample = sample.crop(((sample.width/2), 0, sample.width, sample.height))
    if self.transform is not None:
      sample = self.transform(sample)
    if self.target_transform is not None:
      target = self.target_transform(target)
    return sample, target

In [ ]:
width = 260
height = 260
dimensions = (width, height)

#400 x 500
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split

#Applying Transformation
means,stds = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
train_transforms = transforms.Compose([
                                transforms.Resize(dimensions),
                                transforms.RandomRotation(5),
                                #transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(means,stds),
                                ])

means,stds = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
test_transforms = transforms.Compose([transforms.Resize(dimensions),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means,stds),
                                      ])

mix_test_transforms = transforms.Compose([transforms.Resize((260, 520)),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(means,stds),
                                      ])

batch_size=32
test_set_fold_indx = 4 #0-4


In [ ]:
!rm -R /content/work
file_type = 'coins_kings'
for i in range(5):
  file = 'folds_data_packages/' + file_type +"_fold"+str(i+1)
  load_dataset(file)


folds_paths=[]
for i in range(5):
  folds_paths.append('/content/work/'+file_type+"_fold"+str(i+1))

test_data_kings_both = datasets.ImageFolder(folds_paths[test_set_fold_indx], transform=mix_test_transforms)
test_data_kings = OneCoinSideDataset(folds_paths[test_set_fold_indx],True, transform=test_transforms)
del folds_paths[test_set_fold_indx]
for i in range(len(folds_paths)):
  folds_paths[i] = OneCoinSideDataset(folds_paths[i],True, transform=train_transforms) 

train_data_kings = torch.utils.data.ConcatDataset(folds_paths) 

testloader_kings_both = torch.utils.data.DataLoader(test_data_kings_both, batch_size=batch_size, shuffle=True)
trainloader_kings = torch.utils.data.DataLoader(train_data_kings, batch_size=batch_size, shuffle=True)
testloader_kings = torch.utils.data.DataLoader(test_data_kings, batch_size=batch_size, shuffle=True)

In [ ]:
file_type = 'coins_country'
for i in range(5):
  file = 'folds_data_packages/' + file_type +"_fold"+str(i+1)
  load_dataset(file)

folds_paths=[]
for i in range(5):
  folds_paths.append('/content/work/'+file_type+"_fold"+str(i+1))


train_data_country = OneCoinSideDataset(folds_paths[test_set_fold_indx],False, transform=test_transforms)
del folds_paths[test_set_fold_indx]
for i in range(len(folds_paths)):
  folds_paths[i] = OneCoinSideDataset(folds_paths[i],False, transform=train_transforms) 


test_data_country = torch.utils.data.ConcatDataset(folds_paths) 

trainloader_country = torch.utils.data.DataLoader(train_data_country, batch_size=batch_size, shuffle=True)
testloader_country = torch.utils.data.DataLoader(test_data_country, batch_size=batch_size, shuffle=True)

In [ ]:
def plot_images2(loader, n_batches):

    rows = int(np.sqrt(n_batches*batch_size))+1
    cols = int(np.sqrt(n_batches*batch_size))+1
    fig = plt.figure(figsize = (8, 8))
    loader_iter = iter(loader)
    for i in range(n_batches):
        images, labels = next(loader_iter)
        images = images.permute(0,2,3,1)
        for j in range(batch_size):
          ax = fig.add_subplot(rows, cols, i*batch_size+j+1)
          ax.imshow(images[j])
          ax.set_title(int(labels[j]))
          ax.axis('off')
    #print(loader.dataset.class_to_idx)

plot_images2(testloader,1)
plot_images2(trainloader,1)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer, path):
    save_path = path
    state_dict = torch.load(save_path, map_location=torch.device(device))
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss

def load__SIFT_checkpoint(path):
    save_path = path
    state_dict = torch.load(save_path)
    dict_feature_train = (state_dict['dict_feature_train'])
    dict_feature_test = (state_dict['dict_feature_test'])
    words = state_dict['words']
    print(f'Model loaded from <== {save_path}')
    
    return dict_feature_train, dict_feature_test, words

def dic_of_features_to_X_Y(dict_feature):
  list_feature = []
  for label, label_imgs_list in dict_feature.items():
    for img_feature in label_imgs_list:
      list_feature.append((label,img_feature))
  random.shuffle(list_feature)
  Y, X = zip(*list_feature)
  return X, Y

def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_key, test_val in tests.items():
        class_based[test_key] = [0, 0] 
        for tst in test_val:
            predict_start = 0
            minimum = 0
            key = "a" 
            for train_key, train_val in images.items():
                for train in train_val:
                    if(predict_start == 0):
                        minimum = distance.euclidean(tst, train)
                        key = train_key
                        predict_start += 1
                    else:
                        dist = distance.euclidean(tst, train)
                        if(dist < minimum):
                            minimum = dist
                            key = train_key
            
            if(test_key == key):
                correct_predict += 1
                class_based[test_key][0] += 1
            num_test += 1
            class_based[test_key][1] += 1
    return [num_test, correct_predict, class_based]

def train(model, train_loader, test_loader, num_epochs, criterion, save_name):
    best_val_loss = float("Inf") 
    train_losses = []
    val_losses = []
    cur_step = 0
    best_accuracy = 5
    for epoch in range(num_epochs):
        running_loss = 0.0
        accuracy = eval(model, test_loader)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            save_checkpoint(save_name, model, optimizer, train_losses)
        
        model.train()
        print(" Starting epoch " + str(epoch+1))
        for img1, labels in train_loader:
            # Forward
            img1 = img1.to(device)
            labels = labels.to(device)
            outputs = model(img1)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print("Training loss " + str(train_losses[-1]))
        
    print("Finished Training")  
    return train_losses, val_losses  

# evaluation metrics
def eval(model, test_loader):
    y_test = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        correct = 0
       # print('Starting Iteration')
        count = 0
        for img1, labels in test_loader:
            # Forward
            img1 = img1.to(device)
            outputs = model(img1).to(device)
            #outputs1 = outputs[:,1]
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.tolist())
            y_test.extend(labels.tolist())
           # preds = (outputs1>0.58606374).float()
            #output.cpu().data.numpy().argmax()
           # fpr, tpr, thresh = roc_curve(labels, outputs.cpu().data.numpy()[:,1], pos_label=1)
           # auc_score  = roc_auc_score(labels, outputs.cpu().data.numpy()[:,1])
            labels = labels.to(device)
            for i in range(len(labels)):
              if labels[i] == preds[i]:
                correct = correct+1
              count = count +1

        accuracy = (correct/count)*100
    return print_meassures(y_pred,y_test)

def eval_random(model, test_loader):
    y_test = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        correct = 0
       # print('Starting Iteration')
        count = 0
        for img1, labels in test_loader:
            # Forward
            img1 = img1.to(device)
            outputs = model(img1).to(device)
            #outputs1 = outputs[:,1]
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.tolist())
            y_test.extend(labels.tolist())

            labels = labels.to(device)
            for i in range(len(labels)):
              preds[i] = random.randint(0, 29)
              if labels[i] == preds[i]:
                correct = correct+1
              count = count +1

        accuracy = (correct/count)*100
    print_meassures(y_pred,y_test)
    return accuracy

# evaluation metrics
def eval_mixed(model_country, model_kings, test_loader, combine_params=(True, 1000)):
    y_test = []
    y_pred = []
    with torch.no_grad():
        model_country.eval()
        model_kings.eval()
        correct = 0
       # print('Starting Iteration')
        count = 0
        for img1, labels in test_loader:
            # Forward
            img_kings = img1.to(device)[:,:,:,:260]
            img_country = img1.to(device)[:,:,:,260:]
            outputs_country = F.softmax(model_country(img_country).to(device),dim=1)
            outputs_kings = F.softmax(model_kings(img_kings).to(device),dim=1)
            outputs = combine_scores(outputs_country, outputs_kings, combine_params)
            _, preds = torch.max(outputs, 1)
            y_pred.extend(preds.tolist())
            y_test.extend(labels.tolist())
            labels = labels.to(device)
            for i in range(len(labels)):
              if labels[i] == preds[i]:
                correct = correct+1
              count = count +1
  
        accuracy = (correct/count)*100
        
    return print_meassures(y_pred,y_test)


def combine_scores(outputs_country, outputs_kings, combine_params):
    only_positive_multipliers, multiplier_constant = combine_params
    kings_to_country_index ={0: 4, 1: 4, 2: 2, 3: 3, 4: 4, 5: 3, 6: 4, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 3, 14: 2, 15: 2, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1, 21: 1, 22: 3, 23: 4, 24: 4, 25: 3, 26: 2, 27: 2, 28: 2, 29: 3}
    for i in range(len(outputs_kings)):
      for j in range(30):
        mult = outputs_country[i][kings_to_country_index[j]]*multiplier_constant
        if mult > 1 or (not (only_positive_multipliers)):
          outputs_kings[i][j] *= mult
    output = outputs_kings
    return output

def combine_scores_trivial(outputs_country, outputs_kings):
    kings_to_country_index ={0: 4, 1: 4, 2: 2, 3: 3, 4: 4, 5: 3, 6: 4, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 3, 14: 2, 15: 2, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1, 21: 1, 22: 3, 23: 4, 24: 4, 25: 3, 26: 2, 27: 2, 28: 2, 29: 3}
    for i in range(len(outputs_kings)):
      for j in range(30):
        outputs_kings[i][j] *= (outputs_country[i][kings_to_country_index[j]])*10000
    output = outputs_kings
    return output
#SPRAWDZ CZY SIE MYLI TYLKO W KROLACH WEWNATRZ KLAS, CHCEMY ZMNIEJSZYC POMYLI Z KROLAMI Z INNYCH PANSTW I JEDNOCZESNIE NIE ZWIEKSZYC POMYLKI WEWNATRZ KLAS, ZBADAJ TO, STWORZ CONFUSSION MATRIX 

In [ ]:
from torchvision import models
from efficientnet_pytorch import EfficientNet
model_country = EfficientNet.from_pretrained('efficientnet-b2', num_classes=5).to(device)
model_kings = EfficientNet.from_pretrained('efficientnet-b2', num_classes=30).to(device)

optimizer_country = optim.Adam(model_country.parameters(), lr = 0.01)
optimizer_kings = optim.Adam(model_kings.parameters(), lr = 0.01)
torch.cuda.empty_cache()

num_epochs = 100
criterion = nn.CrossEntropyLoss()

In [ ]:
save_path ='/content/gdrive/MyDrive/magisterka/models/' + 'FOLD5_KINGS_BOTHSIDEACC85.16E82.pt'
#t_loader = trainloader
t_loader = testloader
train_losses = load_checkpoint(model, optimizer, save_path)
#accuracy = eval (model,t_loader)

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("/content/gdrive/MyDrive/magisterka/models/") if isfile(join("/content/gdrive/MyDrive/magisterka/models/", f))]
print(onlyfiles)
kings_models = []
cntry_models = []
both_models = []
SIFT_kings_models = []
SIFT_country_models = []
REST_models = []
for i in onlyfiles:
  if "_kings_" in i: 
      SIFT_kings_models.append(i)
  elif "_cntry_" in i:
      SIFT_country_models.append(i)
  elif "BOTH_FOLD" in i:
      both_models.append(i)
  elif "KINGS_FOLD" in i:
      kings_models.append(i)
  elif "CNTRY_FOLD" in i:
      cntry_models.append(i)
  else:
      REST_models.append(i)

kings_models = sorted(kings_models)
cntry_models = sorted(cntry_models)
both_models = sorted(both_models)
SIFT_kings_models = sorted(SIFT_kings_models)
SIFT_country_models = sorted(SIFT_country_models)
REST_models = sorted(REST_models)
#both_models.remove('BOTH_FOLD3_ACC83.29E51.pt')
print(kings_models)
print(cntry_models)
print(both_models)
print(SIFT_kings_models)
print(SIFT_country_models)
print(REST_models)

In [ ]:
from sklearn import svm
from scipy.spatial import distance

def create_data_loaders(foldid, kings, BOTH=False):
  if kings:
    file_name = 'coins_kings'
  else:
    file_name = 'coins_country'
  folds_paths=[]
  for i in range(5):
    folds_paths.append('/content/work/'+file_name+"_fold"+str(i+1))

  if BOTH:
    test_data_kings_both = datasets.ImageFolder(folds_paths[foldid], transform=test_transforms)
  else:  
    test_data_kings_both = datasets.ImageFolder(folds_paths[foldid], transform=mix_test_transforms)
  test_data_kings = OneCoinSideDataset(folds_paths[foldid],kings, transform=test_transforms)
  del folds_paths[foldid]
  for i in range(len(folds_paths)):
    folds_paths[i] = OneCoinSideDataset(folds_paths[i],kings, transform=train_transforms) 

  train_data_kings = torch.utils.data.ConcatDataset(folds_paths) 

  testloader_kings_both = torch.utils.data.DataLoader(test_data_kings_both, batch_size=batch_size, shuffle=True)
  trainloader_kings = torch.utils.data.DataLoader(train_data_kings, batch_size=batch_size, shuffle=True)
  testloader_kings = torch.utils.data.DataLoader(test_data_kings, batch_size=batch_size, shuffle=True)

  return train_data_kings, test_data_kings, trainloader_kings, testloader_kings, test_data_kings_both, testloader_kings_both

def SIFT_test(save_path_to_sift):
  dict_feature_train, dict_feature_test, words = load__SIFT_checkpoint(save_path_to_sift)
  X, y = dic_of_features_to_X_Y(dict_feature_train)
  clf = svm.SVC(C = 1.0, probability=True)
  clf.fit(X,y)
  X_test, Y_test = dic_of_features_to_X_Y(dict_feature_test)
  preds = clf.predict(X_test)
  results_bowl = knn(dict_feature_train, dict_feature_test) 
  print("Accuracy KNN: " + str((100*results_bowl[1])/results_bowl[0]))
  return print_meassures(preds, Y_test)

def print_meassures(y_pred, y_test):
  if (True):
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Precision:",metrics.precision_score(y_test, y_pred, average='macro'))    
    print("F1 score:",metrics.f1_score(y_test, y_pred, average='macro'))
    cm = metrics.confusion_matrix(y_test, y_pred)
    print("Recall:",metrics.recall_score(y_test, y_pred, average='macro'))
  #  print("Confusion matrix:",cm)
    diagonal = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]).diagonal()

    print(diagonal)
    return metrics.accuracy_score(y_test, y_pred), metrics.precision_score(y_test, y_pred, average='macro'), metrics.f1_score(y_test, y_pred, average='macro'), metrics.recall_score(y_test, y_pred, average='macro'), diagonal,cm
  return 0


In [ ]:

stats = [[[],[],[],[],[]],[[],[],[],[],[]],[[],[],[],[],[]],[[],[],[],[],[]],[[],[],[],[],[]]]
stats_mixes = [[[],[],[],[],[]] for i in range (14)]
#for i, params in enumerate([(True, 1),(True, 10),(True, 100),(True, 1000),(True, 10000),(True, 100000),(True, 1000000),(False, 1),(False, 10),(False, 100),(False, 1000),(False, 10000),(False, 100000),(False, 1000000)]):
for foldnumber in range(5):
  foldnr = foldnumber
  if True:
    #######COUNTRY MODEL
    save_path ='/content/gdrive/MyDrive/magisterka/models/' + cntry_models[foldnr]
    _,_,_,t_loader,_,_ = create_data_loaders(foldnr,False)
    train_losses = load_checkpoint(model_country, optimizer_country, save_path)
    #stats[0][foldnumber] = eval (model_country,t_loader)

  if True:
    #######KING MODEL
    save_path ='/content/gdrive/MyDrive/magisterka/models/' + kings_models[foldnr]
    _,_,_,t_loader,_,testloader_kings_both = create_data_loaders(foldnr,True)
    train_losses = load_checkpoint(model_kings, optimizer_kings, save_path)
    #stats[1][foldnumber] = eval (model_kings,t_loader)
    stats_mixes[0][foldnr] = eval_mixed(model_country,model_kings,testloader_kings_both)

  if False:
    ##### BOTH MODEL
    #'BOTH_FOLD3_ACC81.49E90.pt', 'BOTH_FOLD3_ACC83.29E51.pt'
    save_path ='/content/gdrive/MyDrive/magisterka/models/' + both_models[foldnr]
    _,_,_,t_loader,_,testloader_kings_both = create_data_loaders(foldnr,True, True)
    train_losses = load_checkpoint(model_kings, optimizer_kings, save_path)
    stats[2][foldnumber] = eval (model_kings,testloader_kings_both)
    #print("RANDOM " + str(eval_random (model_kings,testloader_kings_both)))

  if False:
    save_path ='/content/gdrive/MyDrive/magisterka/models/' + SIFT_kings_models[foldnr]
    stats[3][foldnumber] = SIFT_test(save_path)

  if False:
    save_path ='/content/gdrive/MyDrive/magisterka/models/' + SIFT_country_models[foldnr]
    stats[4][foldnumber] = SIFT_test(save_path)


In [ ]:
import pickle
#with open('/content/gdrive/MyDrive/magisterka/EXPERIMENTARRAYPARAMETERS.txt', "wb") as fp:   #Pickling
  #pickle.dump(stats_mixes, fp)
with open('/content/gdrive/MyDrive/magisterka/EXPERIMENTARRAY.txt', "rb") as fp:   # Unpickling
  result_list = pickle.load(fp)

In [ ]:
stats_mixes[0][0][-1]

In [ ]:
#BASIC PARAMETERS EXPERIMENT
for task in range(14):
  sum = 0
  for foldnumber in range(5):
    #print((int(stats_mixes[task][foldnumber][0] * 10000)/100), end =" & ")
    sum += stats_mixes[task][foldnumber][0]
  avg = int((sum/5) * 10000)/100
  print ( avg, end =" & ")

In [ ]:
#MIXED MODEL FOLDS
sum = 0
for foldnumber in range(5):
  print((int(stats_mixes[3][foldnumber][0] * 10000)/100), end =" & ")
  sum += stats_mixes[3][foldnumber][0]
avg = int((sum/5) * 10000)/100
print ( str(avg) + ' \\\\')

#85.16 & 86.34 & 87.06 & 87.26 & 87.21 & 87.11 & 

In [ ]:
#BASIC MODELS ACCURACY PER FOLD AND AVG
# ACC|PREC|F1|RECALL|DIAGONAL|CM
# EFF country | EFF kings | EFF both | SVM kings | SVM country
sum = 0
task = 0
meassure = -1
for foldnumber in range(5):
  #print((int(stats[task][foldnumber][meassure] * 10000)/100), end =" & ")
  sum += int(stats_mixes[task][foldnumber][meassure] * 10000)/100
print ( str((int((sum/5)*100))/100)  + '\\\\')

In [ ]:
#BASIC MODELS ACCURACY PER FOLD AND AVG
for task in [4,0,3,1,2]:
  sum = 0
  for foldnumber in range(5):
    print((int(result_list[task][foldnumber][0] * 10000)/100), end =" & ")
    sum += int(result_list[task][foldnumber][0] * 10000)/100
  print ( str((int((sum/5)*100))/100)  + '\\\\')

In [ ]:
kings_to_country = {'Alexander III': 'Russia',
 'Anna Ioannovna': 'Russia',
 'Augustus III the Sas': 'Poland',
 'Aurelian': 'Roman Empire',
 'Catherine II': 'Russia',
 'Constantine the Great': 'Roman Empire',
 'Elizabeth': 'Russia',
 'Ferdinand II': 'Austria',
 'Ferdinand III': 'Austria',
 'Franz Joseph I': 'Austria',
 'Friedrich Wilhelm I': 'Germany',
 'Friedrich Wilhelm II': 'Germany',
 'Friedrich Wilhelm III': 'Germany',
 'Hadrian': 'Roman Empire',
 'John II Casimir': 'Poland',
 'John III Sobieski': 'Poland',
 'Karl VI': 'Austria',
 'Leopold I': 'Austria',
 'Leopold V': 'Austria',
 'Ludwig I': 'Germany',
 'Ludwig II': 'Germany',
 'Maximilian III Jose': 'Germany',
 'Neron': 'Roman Empire',
 'Nicholas II': 'Russia',
 'Peter I': 'Russia',
 'Philip I': 'Roman Empire',
 'Sigismund I Old': 'Poland',
 'Sigismund III': 'Poland',
 'Stephen Báthory': 'Poland',
 'Trajan': 'Roman Empire'}
kings_to_country_index ={0: 4, 1: 4, 2: 2, 3: 3, 4: 4, 5: 3, 6: 4, 7: 0, 8: 0, 9: 0, 10: 1, 11: 1, 12: 1, 13: 3, 14: 2, 15: 2, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1, 21: 1, 22: 3, 23: 4, 24: 4, 25: 3, 26: 2, 27: 2, 28: 2, 29: 3}

In [ ]:
if False:
  model = model_kings
  testloader = testloader_kings
else:
  model = model_country
  testloader = testloader_country
img = next(iter(testloader))
print(img[1])
img = img[0]

In [ ]:
named_layers = dict(model.named_modules())
for moudle in model.named_modules():
  print(moudle)
#print(named_layers['MBConvBlock'])

In [ ]:
#!pip install ttach
#!pip install grad-cam


from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import argparse

torch.cuda.empty_cache()
#model = resnet50(pretrained=True)
target_layer = named_layers['_conv_head']
input_tensor = img# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model.cpu(), target_layer=target_layer, use_cuda=False)

# If target_category is None, the highest scoring category
# will be used for every image in the batch.
# target_category can also be an integer, or a list of different integers
# for every image in the batch.
target_category = 1

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor.cpu(), target_category=target_category)

In [ ]:
index = 4

MEAN = torch.tensor([0.485, 0.456, 0.406])
STD = torch.tensor([0.229, 0.224, 0.225])

x = img[index] * STD[:, None, None] + MEAN[:, None, None]

grayscale_c = grayscale_cam[index, :]
rgb_img = np.float32(x.permute(1,2,0))
visualization = show_cam_on_image(rgb_img, grayscale_c, use_rgb = True)
#plt.imshow(visualization, cmap='gray')
#plt.imshow(img[index].permute(1,2,0))
plt.figure()
plt.axis('off')

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1,2,figsize=(10,10)) ;
# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0].imshow(visualization)
axarr[0].axis('off')
axarr[1].imshow(x.permute(1,2,0))
axarr[1].axis('off')

In [ ]:
print(train_losses)
#plotting of training and validation loss
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(train_losses, label='Train Loss')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()